In [ ]:
# %config InlineBackend.figure_format = 'retina'

In [ ]:
import pandas as pd
import rtree 
import geopandas
import numpy as np
import shapely

In [ ]:
import matplotlib.pyplot as plt

# Pre-processing data

We will use data from [Chicago, US](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/data). It is already on this directory:

In [ ]:
df = pd.read_csv("./chicago.zip")

In [ ]:
df.head()

We are only interested in specific columns:

In [ ]:
df = df[["Date", "Primary Type", "Latitude", "Longitude"]]

Let's rename these columns to more convenient names:

In [ ]:
df.columns = ["time", "crime", "lat", "lon"]

In [ ]:
df.head()

Note that thera are some `NaN` values. Let's remove them:

In [ ]:
df = df.dropna().copy()

In [ ]:
df.head()

We want to be able to play with date/time here. Let's transform the column `time` into something more useful:

In [ ]:
df.time = pd.to_datetime(df.time)

In [ ]:
df.head()

# Crime concentration analysis

We want only data point within Chicago. 

In [ ]:
# based on https://boundingbox.klokantech.com/
chicago_lon_min, chicago_lon_max = -87.940101,-87.523984
chicago_lat_min, chicago_lat_max = 41.643919, 42.023022

In [ ]:
df = df[(df.lat > chicago_lat_min) & 
        (df.lat < chicago_lat_max) & 
        (df.lon > chicago_lon_min) & 
        (df.lon < chicago_lon_max)]

In [ ]:
plt.figure(figsize=(10, 9))
plt.plot(*zip(*(df[["lon", "lat"]].values)), linestyle="", marker=".", markersize=0.5)
plt.gca().axis('off');

That looks legit.

## Using `geopandas` to create a grid

`geopandas` will help us a lot today! First, we need to create a `GeoDataFrame` using our `DataFrame`: 

In [ ]:
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat))

We could also include the Coordinate Reference System of the data set; we will skip this here (but the interested coder can take a look at the [documentation](https://geopandas.org/docs/reference/api/geopandas.GeoDataFrame.html)). 

Note that `geopandas.GeoDataFrame` converts the latitude/longitude coordinates into a `POINT` geometry:

In [ ]:
gdf.head()

### Creating a grid

We want to create a grid in that all data points are included. Thus, we use the boundaries of our data to define the size of the grid.

In [ ]:
lat_min, lat_max = df.lat.min(), df.lat.max()

lon_min, lon_max = df.lon.min(), df.lon.max()

In [ ]:
lon_min, lat_min, lon_max, lat_max

Next, we should create the cells of this grid: 

In [ ]:
side_size = 15  # number of cells 
cell_size = (lat_max-lat_min)/side_size

In [ ]:
grid = []

In [ ]:
for cell_x in np.arange(lon_min, lon_max, cell_size):
    for cell_y in np.arange(lat_min, lat_max, cell_size):
        cell = shapely.geometry.box(cell_x, cell_y, 
                                    cell_x + cell_size, 
                                    cell_y + cell_size)
        grid.append(cell)

In [ ]:
gdf_grid = geopandas.GeoDataFrame(grid, columns=['geometry'])

In [ ]:
ax = gdf.plot(markersize=1, figsize=(15, 12), cmap='inferno_r')
plt.autoscale(False)
gdf_grid.plot(ax=ax, facecolor="none", edgecolor='pink', linewidth=2, alpha=0.4)
ax.axis("off");

### Counting the number of points in each cell

What we want now is to count the number of points in each cell. 

A quick way to do that is to just perfom a `join` [operation](https://en.wikipedia.org/wiki/Relational_algebra#Joins_and_join-like_operators)! 

With `geopandas`, this operation is trivial (for the interested reader: [documentation](https://geopandas.org/docs/reference/api/geopandas.sjoin.html)):

In [ ]:
gdf_joined = geopandas.sjoin(gdf, gdf_grid, how='left', op='within')

In [ ]:
gdf_joined.head()


<div class="alert alert-info">
WHAT DOES THIS MEAN?
<img width=300 src=https://www.meme-arsenal.com/memes/9842c3db6d1639e09dbe3d55466d76fe.jpg>

</div>

To count, we dissolve (i.e., we `groupby` then aggregate our the groups via an aggregate function).

In [ ]:
dissolve = gdf_joined.dissolve(by="index_right", aggfunc="count")

In [ ]:
dissolve["crime"]

In [ ]:
gdf_grid.loc[dissolve.index, 'count'] = dissolve['crime'].values

In [ ]:
gdf_grid

<div class="alert alert-info">
WHY SO MANY NaNs? 
<img width=300 src=https://www.meme-arsenal.com/memes/9842c3db6d1639e09dbe3d55466d76fe.jpg>

</div>

In [ ]:
gdf_grid.plot(column='count', figsize=(15, 12), cmap='inferno_r', edgecolor="white")
plt.autoscale(False)
plt.gca().axis('off');

# Exercise

<div class="alert alert-success">
    <b>1)</b> Plot the Lorenz curves of crime for different types of crime.
</div>

Hint: The function `cumsum` from `numpy` will help you to calcualte the cumulative sum of a list.

<div class="alert alert-success">
    <b>1.b)</b> What is the impact of grid size on the Lorenz curves?
</div>

<div class="alert alert-success">
    <b>2)</b> Plot a time series of crime in a specific region.
</div>

Hint: Note that you can conveniently use the `groupby` function on the `gdf_joined` object (but you might be an extra variable).